## This is a PoC for building the LLM enabled Chatbot with History/ Open Source

Here I am using LLM with groq based Chat app with history

Note: this will give chat option from LLM, but not from any external enabled sources or it supports any actions.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")


Get the Gemma from ChatGroq

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015CBF137B80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015CBF136EC0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

Compose a Human Message and Invoke it to the Model

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My Name is Ravi, Cloud & ML Technology Manager")])

AIMessage(content="Hello Ravi, it's nice to meet you! \n\nAs a Cloud & ML Technology Manager, what are you working on these days? Any exciting projects or challenges you're tackling?  \n\nI'm eager to learn more about your work.  \n\n", response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 26, 'total_tokens': 83, 'completion_time': 0.115755836, 'prompt_time': 0.002585437, 'queue_time': None, 'total_time': 0.118341273}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7968a3eb-130a-424f-b780-6a4b2cc38d90-0', usage_metadata={'input_tokens': 26, 'output_tokens': 57, 'total_tokens': 83})

Check if it remembers what it responded using the AI Message

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My Name is Ravi, Cloud & ML Technology Manager"),
        AIMessage(content="Hi Ravi, it's nice to meet you! As a Cloud & ML Technology Manager, I imagine you're involved in some exciting projects. What kind of work are you doing these days? I'm always eager to learn more about how people are using AI and cloud technologies.\n"),
        HumanMessage(content="Hi, what is My Name and my profession?"),
    ]
)

AIMessage(content="You told me your name is Ravi, and you are a Cloud & ML Technology Manager.  \n\nIs there anything else you'd like to tell me about yourself or discuss? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 105, 'total_tokens': 146, 'completion_time': 0.082289795, 'prompt_time': 0.016951715, 'queue_time': None, 'total_time': 0.09924151}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2f883e59-8397-490a-bb3d-5e68932166b2-0', usage_metadata={'input_tokens': 105, 'output_tokens': 41, 'total_tokens': 146})

##Message History Background

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store={}

def get_session_hist(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_hist=RunnableWithMessageHistory(model,get_session_history=get_session_hist)

In [7]:
config={"configurable":{"session_id":"chat1"}} # Create a session configurable

Invoke and check if the session is saved

In [8]:
response=with_message_hist.invoke(
    [HumanMessage(content="Hi, My Name is Ravi, Cloud & ML Technology Manager")],
    config=config
)

In [9]:
response.content

"Hi Ravi, it's nice to meet you!\n\nAs a Cloud & ML Technology Manager, what are you working on these days?  \n\nI'm always interested in learning more about the exciting developments in cloud computing and machine learning. 😊\n\n"

all the history is in Session, even if we change the config

In [10]:

config1={"configurable":{"session_id":"chat1"}}
response=with_message_hist.invoke(
    [HumanMessage(content="Hi, what is my Name?")],
    config=config1
)
response.content

'Your name is Ravi. 😊  \n\nI remember that from our earlier conversation.  \n\n\nHow can I help you today, Ravi?\n'

#Adding Prompt templates

    • Prompt Templates help to turn raw user information into a format that the LLM can work with.

    • In this case, the raw user input is just a message, which we are passing to the LLM. 

    • Let's now make that a bit more complicated. 

    • First, let's add in a system message with some custom instructions (but still taking messages as input). 
    
    Next, we'll add in more input besides just the messages.

In [11]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful Servent.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [12]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is RK")]})

AIMessage(content="Hello RK, it's nice to meet you! \n\nHow can I help you today?  😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 36, 'total_tokens': 63, 'completion_time': 0.053348891, 'prompt_time': 0.004345668, 'queue_time': None, 'total_time': 0.057694559}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-41c346e9-fb1f-49fe-bf55-ccccd1d63fc5-0', usage_metadata={'input_tokens': 36, 'output_tokens': 27, 'total_tokens': 63})

In [13]:
with_message_history=RunnableWithMessageHistory(chain,get_session_hist)

In [14]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is RK")],
    config=config
)

response

AIMessage(content="Hello RK, it's nice to meet you! \n\nWhat can I do for you today?  😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 36, 'total_tokens': 64, 'completion_time': 0.056091897, 'prompt_time': 0.004258649, 'queue_time': None, 'total_time': 0.060350546000000005}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2bfcd462-c271-438e-a24a-5462aa5090b6-0', usage_metadata={'input_tokens': 36, 'output_tokens': 28, 'total_tokens': 64})

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is RK.  \n\nI remember that you told me! 😊  How can I help you further?  \n'

In [16]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [17]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is RK")],"language":"German"})
response.content

'Hallo RK, schön dich kennenzulernen! Wie kann ich dir helfen? 😊  \n'

In [18]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_hist,
    input_messages_key="messages"
)

In [19]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am RK")],"language":"German"},
    config=config
)
repsonse.content

'Hallo RK, schön dich kennenzulernen! 👋\n\nWie kann ich dir heute helfen? 😊 \n'

In [20]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Kannada"},
    config=config,
)

In [21]:
response.content

'ನಿಮ್ಮ ಹೆಸರು RK.  😊 \n'

# Conversation History Management
Manage conversation history is very important. Without which, the list of messages will grow unbounded and potentially overflow the context window of the LLM. 
Therefore, it is important to add a step that limits the size of the messages you are passing in.


In [22]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
#trimmer.invoke(messages)

In [25]:
trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, like your favorite ice cream flavor! \n\nWhat's your favorite flavor?  🍦😊\n"

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2".  \n\n\n\nLet me know if you\'d like to try another one! 😊 \n'

In [29]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_hist,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [30]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I don't have access to past conversations or personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  </p>\n"

In [31]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"